# Chatbot Multi-PDF con Memoria Conversacional (Estilo Nativo)

Este notebook demuestra cómo construir un sistema RAG (Retrieval-Augmented Generation) **sin depender de integraciones complejas de LangChain para la conexión con Google**, usando directamente las librerías oficiales.

**Flujo de trabajo:**
1.  **Ingesta**: Cargar PDFs y dividirlos en fragmentos (chunks).
2.  **Vector Store Manual**: Usar `ChromaDB` nativo para guardar embeddings generados por `google-generativeai`.
3.  **Memoria**: Gestión manual del historial de chat.
4.  **RAG**: Buscar, construir prompt y generar respuesta.


Recuerda que puedes obtener tu API Key en: https://aistudio.google.com/api-keys

In [ ]:
# Instalación de librerías
%pip install -q -U google-generativeai chromadb pypdf langchain_community langchain

In [ ]:
import os
import google.generativeai as genai
import chromadb
from chromadb.config import Settings

# Configura tu API Key aquí
GOOGLE_API_KEY = "TU API KEY" 
genai.configure(api_key=GOOGLE_API_KEY)

## 1. Ingesta de Datos
Usamos LangChain SOLO para cargar y dividir el texto, que es donde brilla.

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Cargar PDFs
source_folder = "libros"

# Crea la carpeta si no existe
if not os.path.exists(source_folder):
    os.makedirs(source_folder)
    print(f"La carpeta '{source_folder}' no existía. Se ha creado. ¡Pon tus PDFs ahí!")

loader = PyPDFDirectoryLoader(source_folder)
documents = loader.load()

# 2. Dividir (Chunking)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
chunks_objects = text_splitter.split_documents(documents)

# Convertimos a lista de textos simples para procesarlos manualmente
chunks_text = [doc.page_content for doc in chunks_objects]

print(f"Encontrados {len(documents)} documentos. Creados {len(chunks_text)} fragmentos (chunks).")

## 2. Vector Store Manual (ChromaDB + Google Embeddings)
En lugar de usar la capa de abstracción de LangChain, haremos el ciclo de embeddings "a mano" como en el ejemplo de YouTube.

In [ ]:
# Inicializamos Cliente Persistente de Chroma
# Esto guardará la base de datos en una carpeta local para no re-procesar todo siempre
chroma_client = chromadb.PersistentClient(path="./chroma_db_multi_pdf")
collection_name = "pdf_knowledge_base"

# Borramos la colección anterior si queremos reiniciar limpios (opcional, útil para pruebas)
try:
    chroma_client.delete_collection(name=collection_name)
except:
    pass

collection = chroma_client.create_collection(name=collection_name)

print("Generando embeddings y guardando en ChromaDB...")

# Preparamos listas para el 'batch add' de Chroma
ids = []
embeddings = []
metadatas = []
docs_to_save = []

# Procesamos cada chunk
for i, text in enumerate(chunks_text):
    try:
        # Generamos embedding con la librería oficial de Google
        res = genai.embed_content(
            model="models/text-embedding-004",
            content=text,
            task_type="retrieval_document"
        )
        
        ids.append(str(i))
        embeddings.append(res['embedding'])
        metadatas.append({"source": "pdf_chunk"}) # Puedes agregar más info aquí si quieres
        docs_to_save.append(text)
        
    except Exception as e:
        print(f"Error generando embedding para chunk {i}: {e}")

# Guardamos todo en Chroma
if ids:
    collection.add(
        documents=docs_to_save,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )
    print(f"Guardados {len(ids)} chunks en la Vector Store.")
else:
    print("No se guardó nada (revisa si hay pdf's).")

## 3. Sistema de Chat (Memoria + RAG)
Ahora configuramos el bucle de chat.

In [ ]:
# Configuración del modelo
model = genai.GenerativeModel('models/gemini-2.5-flash')
chat_history = []

def format_history(history, limit=3):
    # Tomamos los últimos 'limit' intercambios
    recent = history[-(limit*2):]
    formatted = ""
    for i in range(0, len(recent), 2):
        if i+1 < len(recent):
            formatted += f"Usuario: {recent[i]}\nIA: {recent[i+1]}\n"
    return formatted

print("\n========================================")
print("¡Chat Multi-PDF listo! Escribe 'exit' para terminar.")
print("========================================\n")

while True:
    question = input("Tú: ")
    if question.lower() in ["salir", "exit", "quit"]:
        print("Hasta la próxima!")
        break
        
    # 1. Recuperación (Retrieval)
    try:
        # Embed de la pregunta
        q_emb = genai.embed_content(
            model="models/text-embedding-004",
            content=question,
            task_type="retrieval_query"
        )['embedding']
        
        # Query a chroma
        results = collection.query(
            query_embeddings=[q_emb],
            n_results=3
        )
        
        # Extraer texto de los resultados
        retrieved_docs = results['documents'][0]
        context_text = "\n---\n".join(retrieved_docs)
        
        # 2. Construcción del Prompt
        history_txt = format_history(chat_history)
        
        prompt = f"""
        Eres un asistente útil que responde preguntas sobre documentos PDF proporcionados.
        Usa el siguiente contexto y el historial de chat para responder.

        [HISTORIAL DE CHAT]
        {history_txt}

        [CONTEXTO RECUPERADO DE LOS PDFs]
        {context_text}

        [PREGUNTA ACTUAL]
        {question}
        
        INSTRUCCIÓN: Responde solo basándote en el contexto. Si no sabes, dilo. Nunca debes alucinar respuestas.
        """
        
        # 3. Generación
        response = model.generate_content(prompt)
        answer = response.text
        
        print(f"Usuario: {question}")
        print(f"\n Modelo RAG: {answer}\n")
        
        # 4. Actualizar Memoria
        chat_history.append(question)
        chat_history.append(answer)

    except Exception as e:
        print(f"Error: {e}")